In [1]:
import xgboost as xgb
import mlflow
import pandas as pd
from sklearn.feature_extraction import DictVectorizer
from sklearn.metrics import root_mean_squared_error, mean_squared_error
import numpy as np

In [2]:
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from hyperopt.pyll import scope

In [3]:
import mlflow 

mlflow.set_tracking_uri("sqlite:///mlflow.db")

mlflow.set_experiment("nyc-taxi-experiment-xgboost")

<Experiment: artifact_location='/workspaces/mlops-zoomcap/02-experiment-tracking/mlruns/2', creation_time=1749109708393, experiment_id='2', last_update_time=1749109708393, lifecycle_stage='active', name='nyc-taxi-experiment-xgboost', tags={}>

In [4]:
def read_dataframe(filename):
    if filename.endswith('.csv'):
        df = pd.read_csv(filename)

        df.tpep_dropoff_datetime = pd.to_datetime(df.tpep_dropoff_datetime)
        df.lpep_pickup_datetime = pd.to_datetime(df.tpep_pickup_datetime)
    elif filename.endswith('.parquet'):
        df = pd.read_parquet(filename)[:73908]

    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    return df

In [5]:
df_train = read_dataframe('/workspaces/mlops-zoomcap/02-experiment-tracking/data/yellow_tripdata_2023-01.parquet')
df_val = read_dataframe('/workspaces/mlops-zoomcap/02-experiment-tracking/data/yellow_tripdata_2023-02.parquet')

In [6]:
df_train['PU_DO'] = df_train['PULocationID'] + '_' + df_train['DOLocationID']
df_val['PU_DO'] = df_val['PULocationID'] + '_' + df_val['DOLocationID']

categorical = ['PU_DO'] #'PULocationID', 'DOLocationID']
numerical = ['trip_distance']

dv = DictVectorizer()

train_dicts = df_train[categorical + numerical].to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)

val_dicts = df_val[categorical + numerical].to_dict(orient='records')
X_val = dv.transform(val_dicts)

target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values

In [7]:
train = xgb.DMatrix(X_train, label=y_train)
valid = xgb.DMatrix(X_val, label=y_val)

In [17]:
def objective(params):
    with mlflow.start_run():
        mlflow.set_tag("model", "xgboost")
        mlflow.log_params(params)
        booster = xgb.train(
            params=params,
            dtrain=train,
            num_boost_round=1000,
            evals=[(valid, 'validation')],
            early_stopping_rounds=50
        )
        y_pred = booster.predict(valid)
        rmse = root_mean_squared_error(y_val, y_pred)
        mlflow.log_metric("rmse", rmse)

    return {'loss': rmse, 'status': STATUS_OK}

In [18]:
search_space = {
    'max_depth': scope.int(hp.quniform('max_depth', 4, 100, 1)),
    'learning_rate': hp.loguniform('learning_rate', -3, 0), # exp(-3), exp(0) - [0.05, 1]
    'reg_alpha': hp.loguniform('reg_alpha', -5, -1),
    'reg_lambda': hp.loguniform('reg_lambda', -6, -1),
    'min_child_weight': hp.loguniform('min_child_weight', -1, 3),
    'objective': 'reg:linear',
    'seed': 42
}

best_result = fmin(
    fn=objective,
    space=search_space,
    algo=tpe.suggest,
    max_evals=50,
    trials=Trials()
)

[0]	validation-rmse:9.33746                           
[1]	validation-rmse:8.62588                           
[2]	validation-rmse:8.06754                           
  0%|          | 0/50 [00:00<?, ?trial/s, best loss=?]

/home/codespace/anaconda3/envs/exp-tracking-env/lib/python3.9/site-packages/xgboost/core.py:158: UserWarning: [07:50:16] WARNING: /workspace/src/objective/regression_obj.cu:227: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)



[3]	validation-rmse:7.63113                           
[4]	validation-rmse:7.29683                           
[5]	validation-rmse:7.03420                           
[6]	validation-rmse:6.83329                           
[7]	validation-rmse:6.68010                           
[8]	validation-rmse:6.56314                           
[9]	validation-rmse:6.46900                           
[10]	validation-rmse:6.39781                          
[11]	validation-rmse:6.34557                          
[12]	validation-rmse:6.30451                          
[13]	validation-rmse:6.27180                          
[14]	validation-rmse:6.24090                          
[15]	validation-rmse:6.21916                          
[16]	validation-rmse:6.20381                          
[17]	validation-rmse:6.19242                          
[18]	validation-rmse:6.18045                          
[19]	validation-rmse:6.17478                          
[20]	validation-rmse:6.16926                          
[21]	valid

/home/codespace/anaconda3/envs/exp-tracking-env/lib/python3.9/site-packages/xgboost/core.py:158: UserWarning: [07:50:49] WARNING: /workspace/src/objective/regression_obj.cu:227: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)



[0]	validation-rmse:8.79374                                                    
[1]	validation-rmse:7.84969                                                    
[2]	validation-rmse:7.24530                                                    
[3]	validation-rmse:6.86547                                                    
[4]	validation-rmse:6.62519                                                    
[5]	validation-rmse:6.47840                                                    
[6]	validation-rmse:6.38550                                                    
[7]	validation-rmse:6.32616                                                    
[8]	validation-rmse:6.28942                                                    
[9]	validation-rmse:6.25946                                                    
[10]	validation-rmse:6.24868                                                   
[11]	validation-rmse:6.23826                                                   
[12]	validation-rmse:6.23379            

...

[2]	validation-rmse:6.21450                                                     
[3]	validation-rmse:6.22911                                                     
[4]	validation-rmse:6.23445                                                     
[5]	validation-rmse:6.22954                                                     
[6]	validation-rmse:6.22991                                                     
[7]	validation-rmse:6.22633                                                     
[8]	validation-rmse:6.21379                                                     
[9]	validation-rmse:6.21210                                                     
[10]	validation-rmse:6.21009                                                    
[11]	validation-rmse:6.20809                                                    
[12]	validation-rmse:6.20815                                                    
[13]	validation-rmse:6.20784                                                    
[14]	validation-rmse:6.20796

/home/codespace/anaconda3/envs/exp-tracking-env/lib/python3.9/site-packages/xgboost/core.py:158: UserWarning: [08:10:47] WARNING: /workspace/src/objective/regression_obj.cu:227: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)



[0]	validation-rmse:8.77212                                                     
[1]	validation-rmse:7.80234                                                     
[2]	validation-rmse:7.17481                                                     
[3]	validation-rmse:6.77968                                                     
[4]	validation-rmse:6.53227                                                     
[5]	validation-rmse:6.37938                                                     
[6]	validation-rmse:6.27937                                                     
[7]	validation-rmse:6.21928                                                     
[8]	validation-rmse:6.18012                                                     
[9]	validation-rmse:6.15141                                                     
[10]	validation-rmse:6.13165                                                    
[11]	validation-rmse:6.12305                                                    
[12]	validation-rmse:6.11783

/home/codespace/anaconda3/envs/exp-tracking-env/lib/python3.9/site-packages/xgboost/core.py:158: UserWarning: [08:10:58] WARNING: /workspace/src/objective/regression_obj.cu:227: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)



[0]	validation-rmse:8.48853                                                     
[1]	validation-rmse:7.46983                                                     
[2]	validation-rmse:6.89476                                                     
[3]	validation-rmse:6.57294                                                     
[4]	validation-rmse:6.39541                                                     
[5]	validation-rmse:6.28950                                                     
[6]	validation-rmse:6.23313                                                     
[7]	validation-rmse:6.20096                                                     
[8]	validation-rmse:6.18106                                                     
[9]	validation-rmse:6.16917                                                     
[10]	validation-rmse:6.16029                                                    
[11]	validation-rmse:6.15269                                                    
[12]	validation-rmse:6.14838

In [19]:
best_result

{'learning_rate': np.float64(0.41524438291981836),
 'max_depth': np.float64(5.0),
 'min_child_weight': np.float64(8.837135957955882),
 'reg_alpha': np.float64(0.17575943381842593),
 'reg_lambda': np.float64(0.050908132731727535)}

In [ ]:
# HOW TO AUTOLOG WITH XGBOOST
best_result = {'learning_rate': np.float64(0.41524438291981836),
 'max_depth': 5,
 'min_child_weight': np.float64(8.837135957955882),
 'reg_alpha': np.float64(0.17575943381842593),
 'reg_lambda': np.float64(0.050908132731727535)}


mlflow.xgboost.autolog()

# ANOTHER POSSIBLE WAY TO SAVE PARAMETERS
# mlflow.log_params(best_result)

booster = xgb.train(
            params=best_result, # WE TAKE THE BEST PERFORMING PARAMETERS
            dtrain=train,
            num_boost_round=1000,
            evals=[(valid, 'validation')],
            early_stopping_rounds=50
        )

# To log the preprocessor
# with open('models/preprocessor.b', 'wb') as f_out:
#     pickle.dump(dv, f_out)
# mflflow.log_artifact('models/preprocessor.b', artifact_path='preprocessor')

# ANOTHER POSSIBLE WAY TO SAVE MODEL
# mlflow.xgboost.log_model(booster, artifact_path = 'models_mlflow')

2025/06/05 09:45:26 INFO mlflow.utils.autologging_utils: Created MLflow autologging run with ID '019b4208d773452c849fefbe0ba65edc', which will track hyperparameters, performance metrics, model artifacts, and lineage information for the current xgboost workflow


[0]	validation-rmse:7.96340
[1]	validation-rmse:6.94058
[2]	validation-rmse:6.50444
[3]	validation-rmse:6.32040
[4]	validation-rmse:6.22535
[5]	validation-rmse:6.18137
[6]	validation-rmse:6.16211
[7]	validation-rmse:6.15218
[8]	validation-rmse:6.14721
[9]	validation-rmse:6.14918
[10]	validation-rmse:6.15116
[11]	validation-rmse:6.13798
[12]	validation-rmse:6.13842
[13]	validation-rmse:6.13636
[14]	validation-rmse:6.13377
[15]	validation-rmse:6.13248
[16]	validation-rmse:6.13036
[17]	validation-rmse:6.13019
[18]	validation-rmse:6.12733
[19]	validation-rmse:6.12392
[20]	validation-rmse:6.12184
[21]	validation-rmse:6.11943
[22]	validation-rmse:6.11937
[23]	validation-rmse:6.11633
[24]	validation-rmse:6.11485
[25]	validation-rmse:6.11442
[26]	validation-rmse:6.11460
[27]	validation-rmse:6.11064
[28]	validation-rmse:6.11029
[29]	validation-rmse:6.10970
[30]	validation-rmse:6.10801
[31]	validation-rmse:6.10557
[32]	validation-rmse:6.10582
[33]	validation-rmse:6.10499
[34]	validation-rmse:6.1

2025/06/05 09:45:45 WARNING mlflow.xgboost: Failed to infer model signature: could not sample data to infer model signature: please ensure that autologging is enabled before constructing the dataset.
2025/06/05 09:45:45 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/home/codespace/anaconda3/envs/exp-tracking-env/lib/python3.9/site-packages/xgboost/core.py:158: UserWarning: [09:45:45] WARNING: /workspace/src/c_api/c_api.cc:1374: Saving model in the UBJSON format as default.  You can use file extension: `json`, `ubj` or `deprecated` to choose between formats."
2025/06/05 09:45:49 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


In [ ]:
# To load a saved model
logged_model = 'runs:/019b4208d773452c849fefbe0ba65edc/model'

# Load model as a PyFuncModel
# loaded_model = mlflow.pyfunc.load_model(logged_model)

# Load model as a XGBoost model
xgboost_model = mlflow.xgboost.load_model(logged_model)

In [ ]:
# You can use the model to predict (this is just an example)
xgboost_model.predict(valid)[:10]

array([ 3.9316607, 35.18125  , 15.94285  , 20.455824 , 23.343243 ,
        5.716869 , 21.131426 ,  4.4266696,  4.9962387,  9.383086 ],
      dtype=float32)